# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Message', engine)


In [4]:
X = df['message']
Y = df.iloc[:, 5:7]

### 2. Write a tokenization function to process your text data

In [5]:
X.iloc[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [6]:
def tokenize(text):
    text = text.lower()
    text = re.sub("[^a-z0-9]", " ", text)
    words = word_tokenize(text) 
    words = filter(lambda x: not(x in stopwords.words("english")), words)
    

    #Reduce words to their stems
    stemmed = [PorterStemmer().stem(w) for w in words]
    
    return stemmed


In [7]:
tokenize(X.iloc[2])

['look', 'someon', 'name']

In [8]:
tokenize(X.iloc[4])

['say', 'west', 'side', 'haiti', 'rest', 'countri', 'today', 'tonight']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [10]:
cv = CountVectorizer(tokenizer=tokenize)
tf = TfidfTransformer()
cl = MultiOutputClassifier(RandomForestClassifier(), n_jobs=8)


pipeline = Pipeline([
    ('preprocess', Pipeline([
        ('count', cv),
        ('tfid', tf)
    ])),
    ('classifier', cl)
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [12]:
# ideas, date, places, numbers, urls

In [13]:
model = pipeline.fit(X_train, y_train)

In [14]:
y_pred = pd.DataFrame(model.predict(X_test), columns=y_test.columns)

In [15]:
Y.mean()

related    0.773650
request    0.170659
dtype: float64

In [16]:
parameters = {
    'scaler__with_mean': [True, False]
    'clf__kernel': ['linear', 'rbf'],
    'clf__C':[1, 10]
}

cv = GridSearchCV(pipeline, param_grid=parameters)


SyntaxError: invalid syntax (<ipython-input-16-65d05109f9a1>, line 3)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
from sklearn.metrics import classification_report

In [18]:
y_test.sort_values(by=y_test.columns[0])

,related,request
3522,0,0
22298,0,0
13985,0,0
18779,0,0
6690,0,0
12914,0,0
16394,0,0
17427,0,0
6622,0,0
5109,0,0


In [19]:
y_pred.describe()

,related,request
count,6554.000000,6554.000000
mean,0.841166,0.087428
std,0.382282,0.282482
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,0.000000
75%,1.000000,0.000000
max,2.000000,1.000000


In [ ]:
repport = classification_report(y_test.iloc[:, 0], y_pred.iloc[:, 0])

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.